In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("ps5.ipynb")

In [2]:
import sqlite3

## Question 1: Basic SQL
In this problem, you'll interact with a toy SQL database using Python's
built-in `sqlite3` package. Documentation can be found at
<https://docs.python.org/3/library/sqlite3.html>. For this problem,
we'll use a popular toy SQLite database, called Chinook, which
represents a digital music collection. See the documentation at <https://github.com/lerocha/chinook-database/blob/master/README.md>
for a more detailed explanation. We'll use the `chinook.sqlite` file:

In [3]:
con = sqlite3.connect('chinook.sqlite')
cur = con.cursor()

**1(a)** (1 pt) Load the database using the Python `sqlite3` package. How many tables are in the database? Save the answer in the variable `n_tables`.

In [4]:
#q = "SELECT count(*) FROM sqlite_master WHERE type = 'table'"
#resp = cur.execute(q)
#test = resp.fetchone()[0]
#print(test)
n_tables = cur.execute("select count(*) from sqlite_master where type = 'table'").fetchone()[0]
#print(n_tables)

In [5]:
#cur.execute("select * from album limit 1").fetchall()
#cur.execute("select * from pragma_table_info('album')").fetchall()
#cur.execute("select * from pragma_table_info('InvoiceLine')").fetchall()
cur.execute("select * from InvoiceLine limit 5").fetchall()

[(1, 1, 2, 0.99, 1),
 (2, 1, 4, 0.99, 1),
 (3, 2, 6, 0.99, 1),
 (4, 2, 8, 0.99, 1),
 (5, 2, 10, 0.99, 1)]

In [6]:
grader.check("q1a")

q1a results: All test cases passed!

**1(b)** (2 pts) What are the names of the tables in the database? Save the answer as
    a set of strings, `table_names`. **Note:** you should write Python `sqlite3`
    code to answer this; don't just look up the answer in the
    documentation!


In [7]:
response  = cur.execute("select name from sqlite_master where type = 'table'")
rows = response.fetchall()
print(rows)

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [8]:
table_names = set()
for name in rows:
    table_names.update(name)
print(table_names)

{'InvoiceLine', 'MediaType', 'PlaylistTrack', 'Genre', 'Track', 'Playlist', 'Album', 'Employee', 'Invoice', 'Customer', 'Artist'}


In [9]:
grader.check("q1b")

q1b results: All test cases passed!

**1(c)** (2 pts) Write a function `albums_starting_with(c)` that takes as an argument a single character `c` and
    returns a list of the primary keys (AlbumIds) of all the albums whose titles
    start with that character. Your function should ignore case, so that
    the inputs "a" and "A" yield the same results. Include error
    checking that raises an error in the event that the input is not a
    single character.


In [10]:
def albums_starting_with(c):
    con = sqlite3.connect('chinook.sqlite')
    cur = con.cursor()
    if len(c) != 1:
        raise ValueError("c must be 1 character")
    else:
        c = c.upper()
        cur.execute("SELECT albumid  from album where SUBSTR(title,1,1) = ?", (c,))
        #cursor.execute('INSERT INTO images VALUES(?)', (img,))
        result = [row[0] for row in cur.fetchall()]
    return result


In [11]:
#c = 'c'
#albums_starting_with(c)

In [12]:
grader.check("q1c")

q1c results: All test cases passed!

**1(d)** (2 pts) Write a function `songs_starting_with(c)` that takes as an argument a single character and
    returns a list of the primary keys (TrackIds) of all the **songs** whose album
    names begin with that letter. Again, your function should ignore
    case and perform error checking as in the previous exercise (again ignoring case).
    **Hint:** you'll need a JOIN statement here. Don't forget that you
    can use the `cursor.description` attribute to find out about tables
    and the names of their columns.


In [13]:
def songs_starting_with(c):
    con = sqlite3.connect('chinook.sqlite')
    cur = con.cursor()
    if len(c) != 1:
        raise ValueError("c must be 1 character")
    else:
        c = c.upper()
        q = """
        SELECT t.TrackId from 
        Album a
        left join Track t on 
        a.AlbumId = t.AlbumId 
        where SUBSTR(a.Title,1,1) = ?
        """
        cur.execute(q, (c,))
        result = [row[0] for row in cur.fetchall()]
    return result


In [14]:
#c = 'a'
#songs_starting_with(c)

In [15]:
grader.check("q1d")

q1d results: All test cases passed!

**1(e)** (2 pts) Write a function `cost_of(c)` that takes as an argument a single character and
    returns the cost of buying every song (consider only the songs that were sold - you need to look into InvoiceLine table) whose album begins with that
    letter. This cost should be based on the tracks' unit prices when it was sold, so
    that the cost of buying a set of tracks is simply the sum of the
    unit prices of all the tracks in the set. Again your function should
    ignore case and perform appropriate error checking. 


In [16]:
#c = 'A'
#con = sqlite3.connect('chinook.sqlite')
#cur = con.cursor()

#q = """
#    SELECT sum(i.UnitPrice) from InvoiceLine i
#    join Track t on 
#    i.TrackId = t.TrackId
#    join Album a on
#    t.AlbumId = a.AlbumId
#    where SUBSTR(a.Title,1,1) = ? and i.UnitPrice is not null
#    """
#result = cur.execute(q, (c,)).fetchall()
#print(result)

In [17]:
def cost_of(c):
    con = sqlite3.connect('chinook.sqlite')
    cur = con.cursor()
    if len(c) != 1:
        raise ValueError("c must be 1 character")
    else:
        c = c.upper()
        q = """
        SELECT DISTINCT sum(i.UnitPrice) from InvoiceLine i
        join Track t on 
        i.TrackId = t.TrackId
        join Album a on
        t.AlbumId = a.AlbumId
        where SUBSTR(a.Title,1,1) = ? and i.UnitPrice is not null
        """
    cur.execute(q, (c,))
    value = [row[0] for row in cur.fetchall()]
    result = value[0]
    return result


In [18]:
#cost_of('A')
#cost_of('bb')
#cost_of('c')
#res = cost_of('a')
#abs(res - 216.82)
#cost_of('a') - 216.82 < 1e-5

In [19]:
grader.check("q1e")

q1e results:
    q1e - 1 result:
        ❌ Test case failed
        Trying:
            res = cost_of('a')
        Expecting nothing
        ok
        Trying:
            assert type(res) == float
        Expecting nothing
        ok
        Trying:
            assert abs(res - 216.82) < 1e-5
        Expecting nothing
        **********************************************************************
        Line 3, in q1e 0
        Failed example:
            assert abs(res - 216.82) < 1e-5
        Exception raised:
            Traceback (most recent call last):
              File "/opt/conda/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q1e 0[2]>", line 1, in <module>
                assert abs(res - 216.82) < 1e-5
                       ^^^^^^^^^^^^^^^^^^^^^^^^
            AssertionError

    q1e - 2 result:
        ✅ Test case passed

    q1e - 3 result:
        ✅ Test case passed

## Question 2: Additional SQL
In this case, we will use the included file `flights.sqlite`

In [20]:
con = sqlite3.connect('flights.sqlite')
cur = con.cursor()

In [21]:
import pandas as pd
query = "SELECT * FROM flights"
df = pd.read_sql_query(query, con)
df.to_csv('flights.csv', index=False)

Each question asks you to write a SQL query that performs a certain operation. For example, if the question asks, "how many flights are there", an answer could be 

> `SELECT COUNT(*) FROM flights`

Some questions refer to a parameter $x$. For these, you should write a query containing a placeholder, which we can run for different values of $x$. For example, if the question asks, "how many flights are there in month $x$?", where $x\in\{1,2,\dots,12\}$ is a numerical month, an answer could be 

> `SELECT COUNT(*) FROM flights WHERE month=?`.

For answers that ask for a single datum, we will run 
```
> cur.execute(your_query)
> result = cur.fetchone()
```
For answers that ask for a table or set of rows, we will use `cur.fetchall(...)` instead of `cur.fetchone(...)`. For answers that accept a parameter, we will use e.g. `cur.fetchone(..., [x])`.

**2(a)** (2 pts) What was the number of scheduled departures on day of the week $x$, where $x$ is the numerical day of the week (Sunday=0, Monday=1, etc.) Your query should return a tuple `(n, x)` where `n` is the number of flights, and `x` is the day of week. 
Hint: Use the time_hour column to get the week

In [22]:
cur.execute(f"pragma table_info ('flights')")
for row in cur:
    print(row)

#print(cur.execute("select * from flights limit 1").fetchall())
print(cur.execute(f"select count(*), strftime('%w', time_hour, 'unixepoch') from flights group by strftime('%w', time_hour, 'unixepoch')").fetchall())

#q = """
#    select count(*), 
#    strftime('%w', time_hour, 'unixepoch') as 'weekday' 
#    from flights 
#    group by strftime('%w', time_hour, 'unixepoch')
#    where 'weekday' = ?
#    """
#cur.execute(q, '1')

(0, 'year', 'INTEGER', 0, None, 0)
(1, 'month', 'INTEGER', 0, None, 0)
(2, 'day', 'INTEGER', 0, None, 0)
(3, 'dep_time', 'INTEGER', 0, None, 0)
(4, 'sched_dep_time', 'INTEGER', 0, None, 0)
(5, 'dep_delay', 'REAL', 0, None, 0)
(6, 'arr_time', 'INTEGER', 0, None, 0)
(7, 'sched_arr_time', 'INTEGER', 0, None, 0)
(8, 'arr_delay', 'REAL', 0, None, 0)
(9, 'carrier', 'TEXT', 0, None, 0)
(10, 'flight', 'INTEGER', 0, None, 0)
(11, 'tailnum', 'TEXT', 0, None, 0)
(12, 'origin', 'TEXT', 0, None, 0)
(13, 'dest', 'TEXT', 0, None, 0)
(14, 'air_time', 'REAL', 0, None, 0)
(15, 'distance', 'REAL', 0, None, 0)
(16, 'hour', 'REAL', 0, None, 0)
(17, 'minute', 'REAL', 0, None, 0)
(18, 'time_hour', 'REAL', 0, None, 0)
[(43796, '0'), (50709, '1'), (50705, '2'), (49998, '3'), (50137, '4'), (50153, '5'), (41278, '6')]


In [23]:
avg_sched_dep = """
    WITH weekday_flights AS ( 
    SELECT CAST(strftime('%w', time_hour, 'unixepoch') AS INTEGER) AS 'weekday'
    FROM flights)
    SELECT COUNT(*), weekday
    FROM weekday_flights
    WHERE weekday = CAST(? AS INTEGER)
    GROUP BY weekday
    """

In [24]:
#cur.execute(avg_sched_dep, [1]).fetchone()

In [25]:
grader.check("q2a")

q2a results: All test cases passed!

**2(b)** (2 pts) Which month had the most scheduled departures? Your query should return tuple containing a single numerical month (for example `(2,)` if the answer is February.)

In [26]:
cur.execute("""
    select count(*), CAST(strftime('%m', time_hour, 'unixepoch') AS INTEGER)
    from flights
    group by CAST(strftime('%m', time_hour, 'unixepoch') AS INTEGER)
    """).fetchall()

[(26953, 1),
 (24936, 2),
 (28886, 3),
 (28353, 4),
 (28783, 5),
 (28231, 6),
 (29428, 7),
 (29381, 8),
 (27529, 9),
 (28905, 10),
 (27200, 11),
 (28191, 12)]

In [27]:
most_deps = """
    with month_flights AS (
    select count(*) as 'total_flights', 
    CAST(strftime('%m', time_hour, 'unixepoch') AS INTEGER) as 'month_int'
    from flights
    group by CAST(strftime('%m', time_hour, 'unixepoch') AS INTEGER)
    )
    select month_int from month_flights
    where total_flights = (select max(total_flights) from month_flights)
    """
    

In [28]:
cur.execute(most_deps).fetchone()

(7,)

In [29]:
grader.check("q2b")

q2b results: All test cases passed!

**2(c)** (3 pts) On how many days was the highest recorded temperature below $x$ degrees?

In [30]:
#cur.execute('select * from flights limit 1').fetchall()
#cur.execute("select name from sqlite_master where type='table';").fetchall()
cur.execute(f"pragma table_info ('weather')").fetchall()
#cur.execute(f'select * from weather limit').fetchall()
#cur.execute(f"pragma table_info ('flights')").fetchall()

[(0, 'origin', 'TEXT', 0, None, 0),
 (1, 'year', 'INTEGER', 0, None, 0),
 (2, 'month', 'INTEGER', 0, None, 0),
 (3, 'day', 'INTEGER', 0, None, 0),
 (4, 'hour', 'INTEGER', 0, None, 0),
 (5, 'temp', 'REAL', 0, None, 0),
 (6, 'dewp', 'REAL', 0, None, 0),
 (7, 'humid', 'REAL', 0, None, 0),
 (8, 'wind_dir', 'REAL', 0, None, 0),
 (9, 'wind_speed', 'REAL', 0, None, 0),
 (10, 'wind_gust', 'REAL', 0, None, 0),
 (11, 'precip', 'REAL', 0, None, 0),
 (12, 'pressure', 'REAL', 0, None, 0),
 (13, 'visib', 'REAL', 0, None, 0)]

In [31]:
#cur.execute("""
#    select * from flights as f
#    join weather as w on 
#    f.origin = w.origin and
#    f.year = w.year and
#    f.month = w.month and
#    f.day = w.day and
#    f.hour = w.hour
#    limit 1
#    """).fetchall()

In [32]:
high_above = """
    with high_temps as (
    select year, month, day, max(temp) as high_temp
    from weather
    group by year, month, day
    )
    select count(*) from high_temps where high_temp < ?
    """

In [33]:
#cur.execute(high_above, (30,)).fetchall()
#cur.execute(high_above, [40]).fetchone()


In [34]:
grader.check("q2c")

q2c results: All test cases passed!

**2(d)** (3 pts) In this dataset, a cancelled flight is one for which `dep_delay` is missing. What percentage of the flights were cancelled overall?

In [35]:
pct_cancelled = """
    select 
    avg(case when dep_delay is null then 1.0 else 0.0 end) 
    from flights
    """

In [36]:
grader.check("q2d")

q2d results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [37]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)